In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [2]:
path = '../raw_data'
files = [f for f in listdir(path) if isfile(join(path, f))] 

In [8]:
list_files = []
for file in range(0, len(files)):
    list_file = pd.read_excel(f'{path}/{files[file]}')
    list_files.append(list_file)
list_all_files = pd.concat(list_files)
list_all_files.Data = pd.to_datetime(list_all_files.Data)
list_files_s = list_all_files.sort_values(by="Data")
df_files = pd.DataFrame(list_files_s)
df_files

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
79,Credito,2019-03-12,Transferência - Liquidação,PRIO3 - PETRO RIO S.A. ...,CLEAR CORRETORA - GRUPO XP,400,23.49,9396.00
42,Credito,2020-01-06,Transferência - Liquidação,BRKM5 - BRASKEM S.A. ...,CLEAR CORRETORA - GRUPO XP,100,28.79,2879.00
43,Debito,2020-01-06,Transferência - Liquidação,COGN3 - COGNA EDUCAÇÃO S.A. ...,CLEAR CORRETORA - GRUPO XP,700,5.25,3675.00
44,Credito,2020-01-06,Transferência - Liquidação,RAPT4 - RANDON S.A. IMPLEMENTOS E PARTICIPACOE...,CLEAR CORRETORA - GRUPO XP,300,8.69,2607.00
71,Credito,2020-02-03,Transferência - Liquidação,JPSA3 - JEREISSATI PARTICIPACOES S.A. ...,CLEAR CORRETORA - GRUPO XP,100,36.83,3683.00
...,...,...,...,...,...,...,...,...
3,Credito,2021-12-11,Transferência - Liquidação,LCAM3 - COMPANHIA DE LOCAÇÃO DAS AMÉRICAS,CLEAR CORRETORA - GRUPO XP,200,22.79,4558.00
4,Credito,2021-12-11,Transferência - Liquidação,MOVI3 - MOVIDA,CLEAR CORRETORA - GRUPO XP,200,16.23,3246.00
5,Credito,2021-12-11,Transferência - Liquidação,RRRP3 - 3R PETROLEUM ÓLEO E GÁS S.A.,CLEAR CORRETORA - GRUPO XP,100,35.36,3536.00
2,Credito,2021-12-11,Transferência - Liquidação,BPAN4 - BANCO PAN S/A,CLEAR CORRETORA - GRUPO XP,400,13.07,5230.00


In [9]:
df_files['Data da compra'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Credito')
df_files['Data da venda'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Debito')
df_files.drop(columns={'Entrada/Saída'}, inplace=True)
df_files = df_files[['Produto', 'Data', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Movimentação', 'Instituição']]
df_stock = df_files[df_files['Movimentação'] == 'Transferência - Liquidação']
df_fixedi_c = df_files[df_files['Movimentação'] == 'Compra']
df_fixedi_v = df_files[df_files['Movimentação'] == 'Venda']
df_fixedi = pd.concat([df_fixedi_c, df_fixedi_v])

In [18]:
# df_stock['saldo de acoes'] = df_stock.groupby('Produto').sum()[[df_stock['Quantidade']]].reset_index()
# df_stock
df_stock_s = df_stock.sort_values(by=['Produto', 'Data'])
df_stock_s['saldo de acoes'] = df_stock_s.groupby(['Produto']).count()[['Quantidade']]
df_stock_s
# items = self.data['order_items']
#         p = items.groupby('order_id').count()[['product_id']]

,Produto,Data,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Movimentação,Instituição,saldo de acoes
48,ABEV3 - AMBEV S.A. ...,2020-05-19,NaT,2020-05-19,100,11.81,1181.00,Transferência - Liquidação,XP INVESTIMENTOS CCTVM S/A,NaN
6,AMER3 - B2W COMPANHIA DIGITAL,2021-10-11,NaT,2021-10-11,72,32.67,2352.24,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP,NaN
31,BBAS3 - BCO BRASIL S.A. ...,2020-03-06,2020-03-06,NaT,50,31.86,1593.00,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP,NaN
17,BBAS3 - BCO BRASIL S.A. ...,2020-06-24,2020-06-24,NaT,50,33.70,1685.00,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP,NaN
7,BBAS3 - BCO BRASIL S.A. ...,2021-05-24,NaT,2021-05-24,210,32.83,6894.30,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP,NaN
...,...,...,...,...,...,...,...,...,...,...
62,VVAR3 - VIA VAREJO S.A. ...,2020-03-20,2020-03-20,NaT,900,4.35,3915.00,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP,NaN
50,VVAR3 - VIA VAREJO S.A. ...,2020-05-05,NaT,2020-05-05,1800,9.00,16200.00,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP,NaN
41,YDUQ3 - YDUQS PARTICIPACOES S.A. ...,2020-03-06,2020-03-06,NaT,40,29.50,1180.00,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP,NaN
14,YDUQ3 - YDUQS PARTICIPACOES S.A. ...,2020-07-28,2020-07-28,NaT,40,32.98,1319.20,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP,NaN


In [ ]:
df_fixedi_c = df_files[df_files['Movimentação'] == 'Compra']
df_fixedi_v = df_files[df_files['Movimentação'] == 'Venda']
df_fixedi = pd.concat([df_fixedi_c, df_fixedi_v])
df_fixedi

In [ ]:
df_fixedi = (df_files[df_files['Movimentação'] == 'Compra']) or (df_files[df_files['Movimentação'] == 'Venda'])
df_fixedi

In [ ]:
test = df_files.sort_values(by="Produto")
test

In [ ]:
df_files['Data da compra'] = df_files['Entrada/Saída'].map({'Credito': df_files['Data']})
df_files

In [ ]:
df_files['Data da compra'] = df_files['Entrada/Saída'].apply({'Credito': df_files['Data']})
df_files

In [ ]:
df['Data da compra'] = df['commissioned'].map(
                   {'Credito':True ,'Debito':False})

In [ ]:
df_files['Data da compra'] = df_files['Data da compra'] == True, df_files.Data)

In [ ]:
df_files

In [ ]:
df_all_files.groupby('Produto')

In [ ]:
list_products = df_all_files.Produto.value_counts()

list_products

In [ ]:
#creating spreadsheet
#nome cnpj data da compra data da venda quantidade valor R$ total lucro ou prejuízo R$ saldo de de ações
b3_sheet = 